In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# 1. Configurações Iniciais
anos = [2023, 2024]
centros_custo = {
    1: "Administrativo", 2: "Marketing", 3: "Jurídico", 4: "TI", 5: "RH",
    6: "Financeiro", 7: "Comercial/Vendas", 8: "Operações", 9: "Facilities", 10: "Projetos Especiais"
}

# Mapeamento simplificado de categorias para o gerador
categorias = {
    1: [1, 2, 3], 2: [4, 5], 3: [6], 4: [7, 8], 5: [9, 10],
    6: [11], 7: [12, 13], 8: [14, 15], 9: [16], 10: [17]
}

# 2. Gerar Fato Orçamento (Mensal e Coerente)
orcamento_data = []
id_orc = 1
for ano in anos:
    for mes in range(1, 13):
        for cc_id, cats in categorias.items():
            for cat_id in cats:
                # Valor base por CC
                valor_base = 5000 if cc_id != 5 else 50000 # RH é mais caro (folha)
                # Erro proposital: valor absurdo ocasional
                if random.random() < 0.01:
                    valor_base = valor_base * 50 
                
                orcamento_data.append([id_orc, ano, mes, cc_id, cat_id, round(valor_base * random.uniform(0.9, 1.1), 2)])
                id_orc += 1

df_orcamento = pd.DataFrame(orcamento_data, columns=['id_orcamento', 'ano', 'mes', 'id_centro_custo', 'id_categoria', 'valor_orcado'])

# 3. Gerar Fato Lançamentos (Diária com Sazonalidade)
lancamentos_data = []
id_lan = 1
data_inicio = datetime(2023, 1, 1)
data_fim = datetime(2024, 12, 31)

atual = data_inicio
while atual <= data_fim:
    # Número de lançamentos por dia (3 a 8)
    for _ in range(random.randint(3, 8)):
        cc_id = random.choice(list(centros_custo.keys()))
        cat_id = random.choice(categorias[cc_id])
        
        # Lógica de valor
        valor = random.uniform(100, 5000)
        if cc_id == 2: # Marketing gasta mais em datas especiais
            if atual.month in [5, 8, 11, 12]:
                valor *= 2.5
        
        # Lógica de Status
        if atual > datetime(2024, 11, 1):
            status = random.choice(["Pago", "Pago", "Aberto", "Pending"])
        else:
            status = random.choice(["Pago", "Paga", "Pago"]) # Erro de grafia proposital

        # Inserção de Erros Propositais
        data_reg = atual
        if random.random() < 0.02: valor = valor * -1 # Valor negativo
        if random.random() < 0.01: data_reg = None # Data nula
        if random.random() < 0.01: cc_id = 99 # ID inexistente
        
        # Campanha de Marketing
        campanha = None
        if cc_id == 2:
            if atual.month == 5: campanha = 1
            elif atual.month == 8: campanha = 2
            elif atual.month == 11: campanha = 3
            elif atual.month == 12: campanha = 4

        lancamentos_data.append([id_lan, data_reg, cc_id, cat_id, random.randint(1, 20), campanha, round(valor, 2), status])
        id_lan += 1
    atual += timedelta(days=1)

df_lancamentos = pd.DataFrame(lancamentos_data, columns=['id_lancamento', 'data_lancamento', 'id_centro_custo', 'id_categoria', 'id_fornecedor', 'id_campanha', 'valor_lancamento', 'status_pagamento'])

# 4. Exportar
df_orcamento.to_csv('fato_orcamento.csv', index=False)
df_lancamentos.to_csv('fato_lancamentos.csv', index=False)
print("Arquivos gerados com sucesso!")